## 신경망 모델 구성하기

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


### 클래스 정의하기

In [10]:
#신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화합니다. \\
#nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
#NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력합니다.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [43]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 10])
Predicted class: tensor([4], device='cuda:0')


### 모델 계층(Layer)

In [45]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
nn.Flatten 계층을 초기화하여 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환


In [46]:

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear
선형 계층은 저장된 weight 와 bias을 사용하여 linear transformation을 적용하는 모듈


In [48]:

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만든다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고, 
 신경망이 다양한 현상을 학습 할 수 있도록 돕는다.

In [49]:

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.8512e-01, -8.9850e-02,  2.0600e-01,  5.7000e-02, -5.3398e-02,
         -1.0299e-01, -8.7196e-04,  3.4155e-01, -7.9040e-03, -1.1206e-01,
         -3.0155e-01,  3.1715e-01, -3.8110e-01,  2.3850e-01,  2.4149e-01,
          3.9368e-03,  3.8307e-01, -2.0077e-02,  2.3209e-02,  1.2901e+00],
        [-2.5953e-01, -6.9039e-02,  1.0713e-01, -1.6291e-01,  8.8644e-02,
          3.8559e-02, -2.5144e-01,  4.6251e-01,  8.1071e-02,  7.6659e-02,
         -5.0051e-01,  2.0023e-01, -2.3218e-01,  3.8176e-01,  2.9298e-01,
         -3.2576e-01,  1.7421e-02,  2.2967e-01,  2.0209e-02,  1.1379e+00],
        [-5.1316e-01, -1.4054e-01,  5.2718e-01, -1.1392e-01,  1.9832e-01,
         -1.5699e-01, -2.4629e-01,  2.3053e-01,  3.4444e-01,  1.7752e-01,
          3.0337e-02,  1.0737e-01,  3.6412e-02,  2.2468e-01,  7.5208e-02,
         -1.6613e-01, -8.5415e-02, -3.0549e-01,  7.7528e-02,  1.7212e+00]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2060, 0.0570, 0.0000,

#### nn.Sequential

nn.Sequential 은 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너를 사용하여 아래와 같은 신경망을 빠르게 만들 수 있다.

In [50]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

#### nn.Softmax
신경망의 마지막 선형 계층은 이 모듈에 전달될 ([-inf, inf])범위의 원시 값인 logits를 반환한다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1]범위로 scale된다. 

In [51]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

### 모델 매개변수
신경망 내부의 많은 계층들을 매개변수화된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다. 모듈을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 파라미터 매소드로 모든 매개변수에 접근할 수 있게된다.

In [52]:
print(f"Model structure: {model} \n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0329, -0.0269, -0.0016,  ...,  0.0049, -0.0263,  0.0245],
        [-0.0045, -0.0195, -0.0235,  ...,  0.0254,  0.0003,  0.0337]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0253,  0.0146], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0244, -0.0155,  0.0385,  ..., -0.0290, -0.0413, -0.0351],
        [ 0.0261, -0.0225, -0.0271,  ..., -0.0348, -0.0214, -0.0250]],
       device='cuda:0', grad_fn=<S